In [1]:
# import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from scipy.stats import ttest_ind
from fractions import Fraction
import scipy.stats as stats
from spread import variance, standard_deviation, zipped_z_scores

2.0
1.4142135623730951
[(-2, -1.414213562373095), (-1, -0.7071067811865475), (0, 0.0), (1, 0.7071067811865475), (2, 1.414213562373095)]


In [3]:
# load in csv files
clean_data_csv = "Output Data Clean.csv"
# read the csv files to data frames
clean_crime_data_df = pd.read_csv(clean_data_csv)

In [4]:
clean_crime_data_df["Population Density"] = clean_crime_data_df ['Population']/clean_crime_data_df ['Land Area']
clean_crime_data_df["People per Officer"] = clean_crime_data_df ['Population']/clean_crime_data_df ['Total Officers']
#clean_crime-data_df["Officer per Capita"] = clean_crime_data_df ['Violent Crime']/clean_crime_data_df ['Total Officers']
clean_crime_data_df.head(9)

,County,County Type,Violent Crime,Murder and Nonnegligent Manslaughter,Forcible Rape,Robbery,Aggravated Assault,Property Crime,Burglary,Larceny - Theft,Motor Vehicle Theft,Total Officers,Population,Land Area,Population Density,People per Officer
0,Alameda,Metropolitan,10308,132,519,4767,4890,47547,11708,26822,9017,2751,1510271,739,2043.668471,548.989822
1,Butte,Metropolitan,751,7,89,153,502,5464,1826,3265,373,260,220000,1636,134.474328,846.153846
2,Contra Costa,Metropolitan,4257,65,187,1525,2480,28491,7546,15494,5451,1612,1049025,715,1467.167832,650.759926
3,El Dorado,Metropolitan,485,3,34,66,382,2614,1112,1460,42,230,181058,1707,106.067955,787.208696
4,Fresno,Metropolitan,4904,63,181,1358,3302,38442,8982,22680,6780,1793,930450,5957,156.194393,518.934746
5,Imperial,Metropolitan,618,3,26,105,484,5659,1867,3106,686,335,174528,4176,41.793103,520.979104
6,Kern,Metropolitan,4875,74,163,1221,3417,30156,9420,15747,4989,1345,839631,8131,103.262944,624.260967
7,Kings,Metropolitan,530,3,29,85,413,2874,791,1722,361,173,152982,1389,110.138229,884.289017
8,Los Angeles,Metropolitan,49447,612,2046,21520,25269,229063,48534,137944,42585,23991,9818605,4057,2420.163914,409.262015


In [5]:
cleaner_crime_data_df = clean_crime_data_df.loc[:, ["County","County Type","Violent Crime","Aggravated Assault","Property Crime","Burglary","Population Density","People per Officer"]]
cleaner_crime_data_df.head()

,County,County Type,Violent Crime,Aggravated Assault,Property Crime,Burglary,Population Density,People per Officer
0,Alameda,Metropolitan,10308,4890,47547,11708,2043.668471,548.989822
1,Butte,Metropolitan,751,502,5464,1826,134.474328,846.153846
2,Contra Costa,Metropolitan,4257,2480,28491,7546,1467.167832,650.759926
3,El Dorado,Metropolitan,485,382,2614,1112,106.067955,787.208696
4,Fresno,Metropolitan,4904,3302,38442,8982,156.194393,518.934746


In [6]:
metro_df = cleaner_crime_data_df.loc[cleaner_crime_data_df["County Type"] =="Metropolitan"]
#Removed Los Angeles as an Outlier 4 times the total crime both violent and property from next highest
metro_df.drop(8, axis=0).head(21)

,County,County Type,Violent Crime,Aggravated Assault,Property Crime,Burglary,Population Density,People per Officer
0,Alameda,Metropolitan,10308,4890,47547,11708,2043.668471,548.989822
1,Butte,Metropolitan,751,502,5464,1826,134.474328,846.153846
2,Contra Costa,Metropolitan,4257,2480,28491,7546,1467.167832,650.759926
3,El Dorado,Metropolitan,485,382,2614,1112,106.067955,787.208696
4,Fresno,Metropolitan,4904,3302,38442,8982,156.194393,518.934746
5,Imperial,Metropolitan,618,484,5659,1867,41.793103,520.979104
6,Kern,Metropolitan,4875,3417,30156,9420,103.262944,624.260967
7,Kings,Metropolitan,530,413,2874,791,110.138229,884.289017
9,Madera,Metropolitan,654,466,3020,1198,70.596631,1026.292517
10,Marin,Metropolitan,492,298,4409,988,485.401923,611.159806


In [7]:
metro_df["Violent Crime"] = metro_df["Violent Crime"].astype(float)

C:\Users\gamew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
stats.pearsonr(metro_df['Violent Crime'], metro_df['Population Density'])

In [ ]:
stats.pearsonr(metro_df['Violent Crime'], metro_df['People per Officer'])

In [9]:
Violent_crimes = metro_df.iloc[:,2]
Pop_dense = metro_df.iloc[:,6]
(t_stat, p) = stats.ttest_ind(metro_df['Violent Crime'], metro_df['Population Density'])
print(f'Ttest results for Violent Crimes to Population Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Violent Crimes to Population Density in Metropolitan Counties
t-statistics is 2.223252157387766.
p-value is 0.02942897970386885.


In [11]:
Aggravated_Assault = metro_df.iloc[0, 35:3]
Pop_dense = metro_df.iloc[0, 35:6]
(t_stat, p) = stats.ttest_ind(metro_df['Aggravated Assault'], metro_df['Population Density'])
print(f'Ttest results for Aggravated Assault to Population Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Aggravated Assault to Population Density in Metropolitan Counties
t-statistics is 1.6590677859191458.
p-value is 0.10157649945478872.


In [12]:
Property_crimes = metro_df.iloc[0, 35:4]
Pop_dense = metro_df.iloc[0, 35:6]
(t_stat, p) = stats.ttest_ind(metro_df['Property Crime'], metro_df['Population Density'])
print(f'Ttest results for Property Crimes to Population Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Property Crimes to Population Density in Metropolitan Counties
t-statistics is 3.6533397073804794.
p-value is 0.0004960518029535803.


In [13]:
Burglary_crimes = metro_df.iloc[0, 35:5]
Pop_dense = metro_df.iloc[0, 35:6]
(t_stat, p) = stats.ttest_ind(metro_df['Burglary'], metro_df['Population Density'])
print(f'Ttest results for Burglary to Population Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Burglary to Population Density in Metropolitan Counties
t-statistics is 3.2404059748628997.
p-value is 0.001827979978591752.


In [14]:
Violent_crimes = metro_df.iloc[0, 35:2]
Officer_dense = metro_df.iloc[0, 35:7]
(t_stat, p) = stats.ttest_ind(metro_df['Violent Crime'], metro_df['People per Officer'])
print(f'Ttest results for Violent Crimes to Officer Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Violent Crimes to Officer Density in Metropolitan Counties
t-statistics is 2.6428238858817723.
p-value is 0.010137179121359486.


In [15]:
Aggravated_Assault = metro_df.iloc[0, 35:3]
Officer_dense = metro_df.iloc[0, 35:7]
(t_stat, p) = stats.ttest_ind(metro_df['Aggravated Assault'], metro_df['People per Officer'])
print(f'Ttest results for Aggravated Assaults to Officer Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Aggravated Assaults to Officer Density in Metropolitan Counties
t-statistics is 2.5630316384698024.
p-value is 0.012528066484258492.


In [16]:
Property_crimes = metro_df.iloc[0, 35:4]
Officer_dense = metro_df.iloc[0, 35:7]
(t_stat, p) = stats.ttest_ind(metro_df['Property Crime'], metro_df['People per Officer'])
print(f'Ttest results for Property Crimes to Officer Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Property Crimes to Officer Density in Metropolitan Counties
t-statistics is 3.7226496346322797.
p-value is 0.00039503258122845905.


In [17]:
Burglary_crimes = metro_df.iloc[0, 35:5]
Officer_dense = metro_df.iloc[0, 35:7]
(t_stat, p) = stats.ttest_ind(metro_df['Burglary'], metro_df['People per Officer'])
print(f'Ttest results for Burglary to Officer Density in Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Burglary to Officer Density in Metropolitan Counties
t-statistics is 3.696625162600391.
p-value is 0.0004304163757680377.


In [18]:
non_metro_df = cleaner_crime_data_df.loc[cleaner_crime_data_df["County Type"] =="Nonmetropolitan"]
non_metro_df.head()

,County,County Type,Violent Crime,Aggravated Assault,Property Crime,Burglary,Population Density,People per Officer
36,Alpine,Nonmetropolitan,11,7,145,24,1.592141,51.086957
37,Amador,Nonmetropolitan,110,87,811,267,64.126263,577.136364
38,Calaveras,Nonmetropolitan,82,59,654,282,44.684314,876.500000
39,Colusa,Nonmetropolitan,48,35,474,140,18.625217,389.436364
40,Del Norte,Nonmetropolitan,110,66,664,305,28.439364,650.227273


In [19]:
Violent_crimes2 = non_metro_df.iloc[0, 20:2]
Pop_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Violent Crime'], non_metro_df['Population Density'])
print(f'Ttest results for Violent Crimes to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Violent Crimes to Population Density in Non-Metropolitan Counties
t-statistics is 3.7168105577179085.
p-value is 0.0006173431600612158.


In [20]:
Aggravated_Assault2 = non_metro_df.iloc[0, 20:3]
Pop_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Aggravated Assault'], non_metro_df['Population Density'])
print(f'Ttest results for Aggravated Assault to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Aggravated Assault to Population Density in Non-Metropolitan Counties
t-statistics is 3.4871313843261156.
p-value is 0.0012011487034154203.


In [21]:
Property_crimes2 = non_metro_df.iloc[0, 20:4]
Pop_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Property Crime'], non_metro_df['Population Density'])
print(f'Ttest results for Property Crimes to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Property Crimes to Population Density in Non-Metropolitan Counties
t-statistics is 4.127425416711992.
p-value is 0.00018079618800811076.


In [22]:
Burglary2 = non_metro_df.iloc[0, 20:5]
Pop_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Burglary'], non_metro_df['Population Density'])
print(f'Ttest results for Burglary to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Burglary to Population Density in Non-Metropolitan Counties
t-statistics is 3.9847443083658654.
p-value is 0.00027840823711192.


In [23]:
Violent_crimes2 = non_metro_df.iloc[0, 20:2]
Officer_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Violent Crime'], non_metro_df['People per Officer'])
print(f'Ttest results for Violent Crimes to Officer Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Violent Crimes to Officer Density in Non-Metropolitan Counties
t-statistics is -6.569943814607853.
p-value is 7.480868003916321e-08.


In [24]:
Aggravated_Assault2 = non_metro_df.iloc[0, 20:3]
Officer_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Aggravated Assault'], non_metro_df['People per Officer'])
print(f'Ttest results for Aggravated Assault to Officer Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Aggravated Assault to Officer Density in Non-Metropolitan Counties
t-statistics is -7.733726677938074.
p-value is 1.8213613750191005e-09.


In [25]:
Property_crimes2 = non_metro_df.iloc[0, 20:4]
Officer_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Property Crime'], non_metro_df['People per Officer'])
print(f'Ttest results for Property Crimes to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Property Crimes to Population Density in Non-Metropolitan Counties
t-statistics is 1.4840456006809348.
p-value is 0.1456347534742499.


In [26]:
Burglary2 = non_metro_df.iloc[0, 20:5]
Officer_dense2 = non_metro_df.iloc[0, 20:6]
(t_stat, p) = stats.ttest_ind(non_metro_df['Burglary'], non_metro_df['People per Officer'])
print(f'Ttest results for Burglary to Population Density in Non-Metropolitan Counties')
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

Ttest results for Burglary to Population Density in Non-Metropolitan Counties
t-statistics is -2.831321126718651.
p-value is 0.0072210471986530606.


In [ ]:
def summarize(title, arr):
    print(f"Summarizing {title}")
    print(f"Variance: {variance(arr)}")
    print(f"Standard Deviation: {standard_deviation(arr)}")
    print(f"Z-Scores: {zipped_z_scores(arr)}")
    print("======")